<a href="https://colab.research.google.com/github/Motunrayo244/EDISS-MP/blob/main/Datacentric_AI(RI)/Voxel51/Voxel51_Objectdetection__detection_mistakes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Finding Detection Mistakes with FiftyOne

Annotations mistakes create an artificial ceiling on the performance of your models. However, finding these mistakes by hand is at least as arduous as the original annotation work! Enter FiftyOne.

In this tutorial, we explore how FiftyOne can be used to help you find mistakes in your object detection annotations. To detect mistakes in classification datasets, check out [this tutorial](https://voxel51.com/docs/fiftyone/tutorials/classification_mistakes.html).

We'll cover the following concepts:

- Loading your existing dataset [into FiftyOne](https://voxel51.com/docs/fiftyone/user_guide/dataset_creation/index.html)
- [Adding model predictions](https://voxel51.com/docs/fiftyone/recipes/adding_detections.html) to your dataset
- Computing insights into your dataset relating to [possible label mistakes](https://voxel51.com/docs/fiftyone/user_guide/brain.html#label-mistakes)
- Visualizing mistakes in the [FiftyOne App](https://voxel51.com/docs/fiftyone/user_guide/app.html)

**So, what's the takeaway?**

FiftyOne can help you find and correct label mistakes in your datasets, enabling you to curate higher quality datasets and, ultimately, train better models!

## Setup

If you haven't already, install FiftyOne:

In [1]:
%%capture
!pip install fiftyone

In order to compute mistakenness, your dataset needs to have two [detections fields](https://voxel51.com/docs/fiftyone/user_guide/using_datasets.html#object-detection), one with your ground truth annotations and one with your model predictions.

In this example, we'll load the [quickstart dataset](https://voxel51.com/docs/fiftyone/user_guide/dataset_zoo/datasets.html#dataset-zoo-quickstart) from the FiftyOne Dataset Zoo, which has ground truth annotations and predictions from a [PyTorch Faster-RCNN model](https://github.com/pytorch/vision/blob/master/torchvision/models/detection/faster_rcnn.py) for a few samples from the COCO dataset.

In [2]:
!wget -nc 'https://cleanlab-public.s3.amazonaws.com/ObjectDetectionBenchmarking/tutorial_obj/example_images.zip' && unzip -q -o example_images.zip

--2024-03-05 02:10:08--  https://cleanlab-public.s3.amazonaws.com/ObjectDetectionBenchmarking/tutorial_obj/example_images.zip
Resolving cleanlab-public.s3.amazonaws.com (cleanlab-public.s3.amazonaws.com)... 52.216.216.233, 52.217.199.121, 52.217.162.9, ...
Connecting to cleanlab-public.s3.amazonaws.com (cleanlab-public.s3.amazonaws.com)|52.216.216.233|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17340957 (17M) [application/zip]
Saving to: ‘example_images.zip’

example_images.zip  100%[===================>]  16.54M  20.4MB/s    in 0.8s    

2024-03-05 02:10:09 (20.4 MB/s) - ‘example_images.zip’ saved [17340957/17340957]



In [3]:
! wget -nc http://images.cocodataset.org/annotations/annotations_trainval2017.zip && unzip -q -o annotations_trainval2017.zip

--2024-03-05 02:10:10--  http://images.cocodataset.org/annotations/annotations_trainval2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 3.5.25.92, 54.231.135.241, 54.231.161.17, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|3.5.25.92|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252907541 (241M) [application/zip]
Saving to: ‘annotations_trainval2017.zip’

annotations_trainva 100%[===================>] 241.19M  39.0MB/s    in 6.4s    

2024-03-05 02:10:16 (37.6 MB/s) - ‘annotations_trainval2017.zip’ saved [252907541/252907541]



In [4]:
import random
import pickle
import fiftyone as fo
import fiftyone.zoo as foz
import json
import requests
import os

Migrating database to v0.23.5


INFO:fiftyone.migrations.runner:Migrating database to v0.23.5


In [5]:
directory = '/content/example_images'
files = os.listdir(directory)
# Filtering only the files.
sample_files = [f for f in files if os.path.isfile(directory+'/'+f)]
pickle.dump(sample_files, open('sample_images.pkl','wb'))

In [6]:
# List of selected image file names
sample_image_list = pickle.load(open("/content/sample_images.pkl", "rb"))

# Path to the COCO annotations file
annotations_path = '/content/annotations/instances_val2017.json'

# Load COCO annotations
with open(annotations_path) as f:
    coco_data = json.load(f)

# Filter images and annotations
info = coco_data['info']
licenses = coco_data['licenses']
filtered_images = [img for img in coco_data['images'] if img['file_name'] in sample_image_list]
image_ids = {img['id']: img for img in filtered_images}
filtered_annotations = [ann for ann in coco_data['annotations'] if ann['image_id'] in image_ids and ann['category_id'] in [1,3,10,47,62]]

# Save filtered annotations to a new JSON file
filtered_data = {
    "info": info,
    "licenses":licenses,
    "images": filtered_images,
    "annotations": filtered_annotations,
    "categories": coco_data['categories']  # Preserve category info
}
with open('annotations/filtered_annotations.json', 'w') as f:
    json.dump(filtered_data, f, indent=4)


In [7]:
# Path to the COCO annotations file
annotations_path = '/content/annotations/filtered_annotations.json'

# Load COCO annotations
with open(annotations_path) as f:
    coco_data = json.load(f)

# Count the number of annotations
num_annotations = len(coco_data['images'])

print(f"Number of annotations: {num_annotations}")


Number of annotations: 118


In [8]:
IMAGES_DIR = '/content/example_images/'
# Load COCO formatted dataset
coco_dataset = fo.Dataset.from_dir(
    dataset_type=fo.types.COCODetectionDataset,
    data_path=IMAGES_DIR,
    labels_path="/content/annotations/filtered_annotations.json",
    include_id=True,
    name = 'cleanlab_coco_sample_dataset'
)

 100% |█████████████████| 118/118 [2.4s elapsed, 0s remaining, 54.7 samples/s]      


INFO:eta.core.utils: 100% |█████████████████| 118/118 [2.4s elapsed, 0s remaining, 54.7 samples/s]      


In [9]:
print(coco_dataset)

Name:        cleanlab_coco_sample_dataset
Media type:  image
Num samples: 118
Persistent:  False
Tags:        []
Sample fields:
    id:            fiftyone.core.fields.ObjectIdField
    filepath:      fiftyone.core.fields.StringField
    tags:          fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:      fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    detections:    fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    segmentations: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    coco_id:       fiftyone.core.fields.IntField


In [10]:
# Print a sample ground truth detection
sample = coco_dataset.first()
print(sample.detections)

<Detections: {
    'detections': [
        <Detection: {
            'id': '65e67f29f724d5c225b8900d',
            'attributes': {},
            'tags': [],
            'label': 'person',
            'bounding_box': [
                0.701359375,
                0.3757460317460317,
                0.07953125,
                0.3899365079365079,
            ],
            'mask': None,
            'confidence': None,
            'index': None,
            'supercategory': 'person',
            'iscrowd': 0,
        }>,
    ],
}>


Let's start by visualizing the dataset in the [FiftyOne App](https://voxel51.com/docs/fiftyone/user_guide/app.html):

[ ]
# Inspect some samples and detections
# This is the first detection of the first sample
print(mistake_view.first().ground_truth.detections[0])
<Detection: {
    'id': '5f452487ef00e6374aad2744',
    'attributes': BaseDict({}),
    'tags': BaseList([]),
    'label': 'tv',
    'bounding_box': BaseList([
        0.002746666666666667,
        0.36082,
        0.24466666666666667,
        0.3732,
    ]),
    'mask': None,
    'confidence': None,
    'index': None,
    'area': 16273.3536,
    'iscrowd': 0.0,
    'mistakenness': 0.005771428346633911,
    'mistakenness_loc': 0.16955941131917984,
}>

In [12]:
# Open the dataset in the App
session = fo.launch_app(coco_dataset)


Welcome to

███████╗██╗███████╗████████╗██╗   ██╗ ██████╗ ███╗   ██╗███████╗
██╔════╝██║██╔════╝╚══██╔══╝╚██╗ ██╔╝██╔═══██╗████╗  ██║██╔════╝
█████╗  ██║█████╗     ██║    ╚████╔╝ ██║   ██║██╔██╗ ██║█████╗
██╔══╝  ██║██╔══╝     ██║     ╚██╔╝  ██║   ██║██║╚██╗██║██╔══╝
██║     ██║██║        ██║      ██║   ╚██████╔╝██║ ╚████║███████╗
╚═╝     ╚═╝╚═╝        ╚═╝      ╚═╝    ╚═════╝ ╚═╝  ╚═══╝╚══════╝ v0.23.5

If you're finding FiftyOne helpful, here's how you can get involved:

|
|  ⭐⭐⭐ Give the project a star on GitHub ⭐⭐⭐
|  https://github.com/voxel51/fiftyone
|
|  🚀🚀🚀 Join the FiftyOne Slack community 🚀🚀🚀
|  https://slack.voxel51.com
|



INFO:fiftyone.core.session.session:
Welcome to

███████╗██╗███████╗████████╗██╗   ██╗ ██████╗ ███╗   ██╗███████╗
██╔════╝██║██╔════╝╚══██╔══╝╚██╗ ██╔╝██╔═══██╗████╗  ██║██╔════╝
█████╗  ██║█████╗     ██║    ╚████╔╝ ██║   ██║██╔██╗ ██║█████╗
██╔══╝  ██║██╔══╝     ██║     ╚██╔╝  ██║   ██║██║╚██╗██║██╔══╝
██║     ██║██║        ██║      ██║   ╚██████╔╝██║ ╚████║███████╗
╚═╝     ╚═╝╚═╝        ╚═╝      ╚═╝    ╚═════╝ ╚═╝  ╚═══╝╚══════╝ v0.23.5

If you're finding FiftyOne helpful, here's how you can get involved:

|
|  ⭐⭐⭐ Give the project a star on GitHub ⭐⭐⭐
|  https://github.com/voxel51/fiftyone
|
|  🚀🚀🚀 Join the FiftyOne Slack community 🚀🚀🚀
|  https://slack.voxel51.com
|



When working with FiftyOne datasets that contain a field with `Detections`, you can create a [patches view](https://voxel51.com/docs/fiftyone/user_guide/app.html#viewing-object-patches) both through Python and directly in the FiftyOne App to view each detection as a separate sample.

In [13]:
patches_view = coco_dataset.to_patches("detections")
print(patches_view)

Dataset:     cleanlab_coco_sample_dataset
Media type:  image
Num patches: 326
Patch fields:
    id:         fiftyone.core.fields.ObjectIdField
    sample_id:  fiftyone.core.fields.ObjectIdField
    filepath:   fiftyone.core.fields.StringField
    tags:       fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:   fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    detections: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detection)
View stages:
    1. ToPatches(field='detections', config=None)


Let's open the App and click the [patches button](https://voxel51.com/docs/fiftyone/user_guide/app.html#viewing-object-patches), then select `ground_truth` to create the same view that we created above.

In [20]:
session = fo.launch_app(coco_dataset)

## Adding Model Prediction

In [16]:
import torch
import torchvision
from torchvision.models.detection import FasterRCNN_ResNet50_FPN_Weights

# Run the model on GPU if it is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load a pre-trained Faster R-CNN model
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT)
model.to(device)
model.eval()

print("Model ready")

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100%|██████████| 160M/160M [00:01<00:00, 129MB/s]


Model ready


In [22]:
from PIL import Image
from torchvision.transforms import functional as func

import fiftyone as fo

# Get class list
classes = coco_dataset.default_classes

# Add predictions to samples
with fo.ProgressBar() as pb:
    for sample in pb(coco_dataset):
        # Load image
        image = Image.open(sample.filepath)
        image = func.to_tensor(image).to(device)
        c, h, w = image.shape

        # Perform inference
        preds = model([image])[0]
        labels = preds["labels"].cpu().detach().numpy()
        scores = preds["scores"].cpu().detach().numpy()
        boxes = preds["boxes"].cpu().detach().numpy()

        # Convert detections to FiftyOne format
        detections = []
        for label, score, box in zip(labels, scores, boxes):
          if label in [1,3,10,47,62]:
            # Convert to [top-left-x, top-left-y, width, height]
            # in relative coordinates in [0, 1] x [0, 1]
            x1, y1, x2, y2 = box
            rel_box = [x1 / w, y1 / h, (x2 - x1) / w, (y2 - y1) / h]

            detections.append(
                fo.Detection(
                    label=classes[label],
                    bounding_box=rel_box,
                    confidence=score
                )
            )

        # Save predictions to dataset
        sample["predictions"] = fo.Detections(detections=detections)
        sample.save()

 100% |█████████████████| 118/118 [22.6m elapsed, 0s remaining, 0.1 samples/s]    


INFO:eta.core.utils: 100% |█████████████████| 118/118 [22.6m elapsed, 0s remaining, 0.1 samples/s]    


In [26]:
print(coco_dataset)

Name:        cleanlab_coco_sample_dataset
Media type:  image
Num samples: 118
Persistent:  False
Tags:        []
Sample fields:
    id:            fiftyone.core.fields.ObjectIdField
    filepath:      fiftyone.core.fields.StringField
    tags:          fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:      fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    detections:    fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    segmentations: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    coco_id:       fiftyone.core.fields.IntField
    predictions:   fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)


In [27]:
predicted_view = coco_dataset.to_patches("predictions")
print(predicted_view)

Dataset:     cleanlab_coco_sample_dataset
Media type:  image
Num patches: 1078
Patch fields:
    id:          fiftyone.core.fields.ObjectIdField
    sample_id:   fiftyone.core.fields.ObjectIdField
    filepath:    fiftyone.core.fields.StringField
    tags:        fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:    fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    predictions: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detection)
View stages:
    1. ToPatches(field='predictions', config=None)


In [28]:
session = fo.launch_app(coco_dataset)

## Compute mistakenness

Now we're ready to assess the mistakenness of the ground truth detections.

We can do so by running the [compute_mistakenness()](https://voxel51.com/docs/fiftyone/api/fiftyone.brain.html#fiftyone.brain.compute_mistakenness) method from the FiftyOne Brain:

In [29]:
import fiftyone.brain as fob

# Compute mistakenness of annotations in `ground_truth` field using
# predictions from `predictions` field as point of reference
fob.compute_mistakenness(coco_dataset, "predictions", label_field="detections")

Evaluating detections...


INFO:fiftyone.utils.eval.detection:Evaluating detections...


 100% |█████████████████| 118/118 [8.1s elapsed, 0s remaining, 10.5 samples/s]      


INFO:eta.core.utils: 100% |█████████████████| 118/118 [8.1s elapsed, 0s remaining, 10.5 samples/s]      


Computing mistakenness...


INFO:fiftyone.brain.internal.core.mistakenness:Computing mistakenness...


 100% |█████████████████| 118/118 [3.8s elapsed, 0s remaining, 18.7 samples/s]      


INFO:eta.core.utils: 100% |█████████████████| 118/118 [3.8s elapsed, 0s remaining, 18.7 samples/s]      


Mistakenness computation complete


INFO:fiftyone.brain.internal.core.mistakenness:Mistakenness computation complete


The above method populates a number of fields on the samples of our dataset as well as the ground truth and predicted objects:

New ground truth object attributes (in `detection` field):

- `mistakenness` (float): A measure of the likelihood that a ground truth object's label is incorrect
- `mistakenness_loc`: A measure of the likelihood that a ground truth object's localization (bounding box) is inaccurate
- `possible_spurious`: Ground truth objects that were not matched with a predicted object and are deemed to be likely spurious annotations will have this attribute set to True

New predicted object attributes (in `predictions` field):

- `possible_missing`: If a highly confident prediction with no matching ground truth object is encountered, this attribute is set to True to indicate that it is a likely missing ground truth annotation

Sample-level fields:

- `mistakenness`: The maximum mistakenness of the ground truth objects in each sample
- `possible_spurious`: The number of possible spurious ground truth objects in each sample
- `possible_missing`: The number of possible missing ground truth objects in each sample

## Analyzing the results

Let's use FiftyOne to investigate the results.

First, let's show the samples with the most likely annotation mistakes:

In [30]:
from fiftyone import ViewField as F

# Sort by likelihood of mistake (most likely first)
mistake_view = coco_dataset.sort_by("mistakenness", reverse=True)

# Print some information about the view
print(mistake_view)

Dataset:     cleanlab_coco_sample_dataset
Media type:  image
Num samples: 118
Sample fields:
    id:                fiftyone.core.fields.ObjectIdField
    filepath:          fiftyone.core.fields.StringField
    tags:              fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:          fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    detections:        fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    segmentations:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    coco_id:           fiftyone.core.fields.IntField
    predictions:       fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    mistakenness:      fiftyone.core.fields.FloatField
    possible_missing:  fiftyone.core.fields.IntField
    possible_spurious: fiftyone.core.fields.IntField
View stages:
    1. SortBy(field_or_expr='mistakenness', reverse=True, create_index=True)


In [32]:
# Inspect some samples and detections
# This is the first detection of the first sample
print(mistake_view.first().detections.detections[0])

<Detection: {
    'id': '65e67f2af724d5c225b89170',
    'attributes': {},
    'tags': [],
    'label': 'chair',
    'bounding_box': [
        0.0014375,
        0.32675,
        0.07876562499999999,
        0.17024999999999998,
    ],
    'mask': None,
    'confidence': None,
    'index': None,
    'supercategory': 'furniture',
    'iscrowd': 0,
    'mistakenness': 0.9203526079654694,
    'mistakenness_loc': 0.7812364491995805,
}>


Let's use the App to visually inspect the results:

In [33]:
# Open new App window
session.show()

In [34]:
# Show the samples we processed in rank order by the mistakenness
session.view = mistake_view

Another useful query is to find all objects that have a high mistakenness, lets say > 0.50:

In [36]:
from fiftyone import ViewField as F

session.view = coco_dataset.filter_labels("detections", F("mistakenness") > 0.50)

Looking through the results, we see some annotations that may be incorrect. For example, in the image below the `goat` is labeled as a `sheep`.

We can use a similar workflow to look at objects that may be localized poorly:

In [38]:
session.view = coco_dataset.filter_labels("detections", F("mistakenness_loc") > 0.50)

One of the examples that popped up from this query is shown below. The bounding box around the person on the left side of the image is shifted too far to the right.

The `possible_missing` field can also be useful to sort by to find instances of incorrect annotations.

Similarly, `possible_spurious` can be used to find objects that the model detected that may have been missed by annotators.

In [39]:
session.view = coco_dataset.match(F("possible_missing") > 0)

An example that showed up from this search is shown above. There is an `apple` that was not annotated that the model detected.

## Tagging and resolution

Any label or collection of labels can be tagged at any time in the sample grid or expanded sample view. In the expanded sample view, individual samples can be selected by clicking on them in the media player. We can, for example, tag this `apple` prediction as `missing` and any other predictions without an associated ground truth detection.

Labels with specific tags can then be selected with [select_labels()](https://voxel51.com/docs/fiftyone/api/fiftyone.core.collections.html?highlight=select_labels#fiftyone.core.collections.SampleCollection.select_labels) stage and sent off to assist in improving the annotations with your annotation provided of choice. FiftyOne currently offers integrations for both [Labelbox](https://voxel51.com/docs/fiftyone/api/fiftyone.utils.labelbox.html) and [Scale](https://voxel51.com/docs/fiftyone/api/fiftyone.utils.scale.html).

In [40]:
# A dataset can be filtered to only contain labels with certain tags
# Helpful for isolating labels with issues and sending off to an annotation provider
missing_ground_truth = coco_dataset.select_labels(tags="missing")

**REMEMBER**: Since you are using model predictions to guide the mistakenness process, the better your model, the more accurate the mistakenness suggestions. Additionally, using logits of confidence scores will also provide better results.

We used Faster-RCNN in this example which is quite a few years old. Using EfficientDet D7 provided much better results. For example, it was easily able to find this `snowboard` labeled as `skis`:

![skis](https://github.com/voxel51/fiftyone/blob/v0.23.5/docs/source/tutorials/images/det_mistakenness_6.png?raw=1)

In [41]:
session.freeze() # screenshot the active App for sharing